In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# pandas and numpy
import pandas as pd
import numpy as pd

#gcp modules
from google.cloud import bigquery

#Viz
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
project_id = ''
client = bigquery.Client()

Using Kaggle's public dataset BigQuery integration.


In [4]:
owner_project_id = "bigquery-public-data"

# Construct a reference to the "fda food dataset"
project_dataset = 'fda_food'

dataset_ref = client.dataset(project_dataset, project=owner_project_id)

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# get all the tables in  the dataset
tables = list(client.list_tables(dataset))

# print names of all tables in dataset
for table in tables:
    print(table.table_id)

food_enforcement
food_events


We have two tables: enforcement and events. let's focus on enforcement.

We'll start by looking at the table content.

In [5]:
dataset_ref = client.dataset("fda_food", project = 'bigquery-public-data')
table_ref = dataset_ref.table("food_enforcement")

In [6]:
client.list_rows(table_ref, max_results = 5).to_dataframe()

,classification,center_classification_date,report_date,postal_code,termination_date,recall_initiation_date,recall_number,city,event_id,distribution_pattern,...,initial_firm_notification,status,product_type,country,product_description,code_info,address_1,address_2,product_quantity,more_code_info
0,Class II,2016-11-08,2016-11-16,00637,2017-07-03,2016-07-13,F-0347-2017,Sabana Grande,74836,USA only: Florida,...,E-Mail,Terminated,Food,United States,Iberia Codfish Fritter Mix,"Lot #16076, Exp. 3/16/17 and Lot # 16089, Exp....",Zona Industrial Machuchal,Ave. Quilinchini #61,282 boxes of 12 (4.5 oz) pouches each,nan
1,Class II,2018-11-06,2018-11-14,00757,2019-04-24,2018-10-04,F-0372-2019,Santa Isabel,81245,US Consignees only. No government nor foreign ...,...,E-Mail,Terminated,Food,United States,Orange plus Citrus Beverage from Concentrate,Lot 81 14055 T Expiration: Nov 25 18,"Road #545, Km 0.3",nan,"1,010 cases (8 units ea)",nan
2,Class III,2024-01-09,2024-01-17,00911-2317,2024-05-01,2023-12-07,F-0710-2024,San Juan,93627,Domestic distribution only throughout Puerto Rico,...,E-Mail,Terminated,Food,United States,Brand Name: SUPREX Plant Based Nutrition. Prod...,Manufacturer Lot #2633; Expiration Date: 07/25,227 Calle Betances,nan,1999 units,nan
3,Class III,2021-05-07,2021-05-19,00924,2021-10-08,2021-04-20,F-0484-2021,Rio Piedras,87758,Domestic distribution only-PR,...,E-Mail,Terminated,Food,United States,"BTREX PLUS, Dietary Supplement 100 Caplets",Lot # 33277 Exp date:12/22,Calle De Diego # 488 Altos,nan,311 Bottles,nan
4,Class III,2021-05-19,2021-05-26,00924,2021-10-08,2020-07-13,F-0540-2021,Rio Piedras,87759,Domestic distribution only-PR,...,E-Mail,Terminated,Food,United States,"BTREX PLUS, Dietary Supplement 2 Caplets, UPC ...","Lot# 32179, Exp. 9/2022 Lot# 33278, Exp. 12/2022",Calle De Diego # 488 Altos,nan,lots #32179 1405 and Lot #33278 - 0.,nan


In [7]:
food = client.list_rows(table_ref, max_results = 5).to_dataframe()
food

,classification,center_classification_date,report_date,postal_code,termination_date,recall_initiation_date,recall_number,city,event_id,distribution_pattern,...,initial_firm_notification,status,product_type,country,product_description,code_info,address_1,address_2,product_quantity,more_code_info
0,Class II,2016-11-08,2016-11-16,00637,2017-07-03,2016-07-13,F-0347-2017,Sabana Grande,74836,USA only: Florida,...,E-Mail,Terminated,Food,United States,Iberia Codfish Fritter Mix,"Lot #16076, Exp. 3/16/17 and Lot # 16089, Exp....",Zona Industrial Machuchal,Ave. Quilinchini #61,282 boxes of 12 (4.5 oz) pouches each,nan
1,Class II,2018-11-06,2018-11-14,00757,2019-04-24,2018-10-04,F-0372-2019,Santa Isabel,81245,US Consignees only. No government nor foreign ...,...,E-Mail,Terminated,Food,United States,Orange plus Citrus Beverage from Concentrate,Lot 81 14055 T Expiration: Nov 25 18,"Road #545, Km 0.3",nan,"1,010 cases (8 units ea)",nan
2,Class III,2024-01-09,2024-01-17,00911-2317,2024-05-01,2023-12-07,F-0710-2024,San Juan,93627,Domestic distribution only throughout Puerto Rico,...,E-Mail,Terminated,Food,United States,Brand Name: SUPREX Plant Based Nutrition. Prod...,Manufacturer Lot #2633; Expiration Date: 07/25,227 Calle Betances,nan,1999 units,nan
3,Class III,2021-05-07,2021-05-19,00924,2021-10-08,2021-04-20,F-0484-2021,Rio Piedras,87758,Domestic distribution only-PR,...,E-Mail,Terminated,Food,United States,"BTREX PLUS, Dietary Supplement 100 Caplets",Lot # 33277 Exp date:12/22,Calle De Diego # 488 Altos,nan,311 Bottles,nan
4,Class III,2021-05-19,2021-05-26,00924,2021-10-08,2020-07-13,F-0540-2021,Rio Piedras,87759,Domestic distribution only-PR,...,E-Mail,Terminated,Food,United States,"BTREX PLUS, Dietary Supplement 2 Caplets, UPC ...","Lot# 32179, Exp. 9/2022 Lot# 33278, Exp. 12/2022",Calle De Diego # 488 Altos,nan,lots #32179 1405 and Lot #33278 - 0.,nan


In [8]:
column_names = list(food.columns.values)
column_names

['classification',
 'center_classification_date',
 'report_date',
 'postal_code',
 'termination_date',
 'recall_initiation_date',
 'recall_number',
 'city',
 'event_id',
 'distribution_pattern',
 'recalling_firm',
 'voluntary_mandated',
 'state',
 'reason_for_recall',
 'initial_firm_notification',
 'status',
 'product_type',
 'country',
 'product_description',
 'code_info',
 'address_1',
 'address_2',
 'product_quantity',
 'more_code_info']

In [9]:
client.list_rows(table_ref, max_results=10).to_dataframe().transpose

<bound method DataFrame.transpose of   classification center_classification_date report_date postal_code  \
0       Class II                 2016-11-08  2016-11-16       00637   
1       Class II                 2018-11-06  2018-11-14       00757   
2      Class III                 2024-01-09  2024-01-17  00911-2317   
3      Class III                 2021-05-07  2021-05-19       00924   
4      Class III                 2021-05-19  2021-05-26       00924   
5      Class III                 2021-05-19  2021-05-26       00924   
6       Class II                 2016-10-03  2016-10-12       00962   
7       Class II                 2016-10-03  2016-10-12       00962   
8       Class II                 2016-10-03  2016-10-12       00962   
9       Class II                 2016-10-03  2016-10-12       00962   

  termination_date recall_initiation_date recall_number           city  \
0       2017-07-03             2016-07-13   F-0347-2017  Sabana Grande   
1       2019-04-24             20